In [1]:
# Set constants for environment
import os
import sys

PROJECT_NAME = 'lvgl-ui-detector'

IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL_CONTAINER = 'HOMELAB' in os.environ
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/project') # Mount Google Drive
    root_dir = os.path.join("/content/drive/project")
elif IN_LOCAL_CONTAINER:
    root_dir = os.path.join(os.environ['HOME'], 'project')
else:
    root_dir = os.path.curdir

In [2]:
if IN_COLAB and 'ultralytics' not in sys.modules:
    %pip install ultralytics
%pip show ultralytics

Name: ultralytics
Version: 8.1.45
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /opt/conda/lib/python3.10/site-packages
Editable project location: /usr/src/ultralytics
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Data preparation

In [3]:
import glob
if IN_COLAB:
    repository_dir = os.path.join(root_dir, PROJECT_NAME)
    dataset_dir = os.path.join(root_dir, 'datasets')
    dataset_files = glob.glob(os.path.join(dataset_dir, '*.yaml'))
else:
    datasets_dir = os.path.join(root_dir, 'tmp')
    dataset_files = glob.glob(os.path.join(datasets_dir, '*.yaml'))

print(f"Datasets: {dataset_files}")

Datasets: ['./tmp/designs.yaml', './tmp/random_widgets.yaml', './tmp/random_uis.yaml']


# Model training

In [5]:
from clearml import Task
from ultralytics import YOLO 

# Create a ClearML Task
task = Task.init(
    project_name="LVGL UI Detector",
    task_name="train"
)

# Load a model
model_variant = "yolov8n"
# Log "model_variant" parameter to task
task.set_parameter("model_variant", model_variant)

# Load the YOLOv8 model
model = YOLO(f'{model_variant}.pt') 

# Put all YOLOv8 arguments in a dictionary and pass it to ClearML
# When the arguments are later changed in UI, they will be overridden here!
args = dict(data=dataset_files[2], epochs=100, imgsz=480)
task.connect(args)

# Train the model 
# If running remotely, the arguments may be overridden by ClearML if they were changed in the UI
results = model.train(**args)

2024-04-08 14:45:26,155 - clearml.model - WARNING - Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely
Ultralytics YOLOv8.1.45 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./tmp/random_uis.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visuali

train: Scanning /usr/src/datasets/random_uis/labels/train... 65 images, 0 backgrounds, 0 corrupt: 100%|██████████| 65/65 [00:00<00:00, 465.51it/s]

train: New cache created: /usr/src/datasets/random_uis/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /usr/src/datasets/random_uis/labels/val... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 545.13it/s]

...

                   all          9         44      0.443      0.654      0.624        0.4



100 epochs completed in 0.062 hours.
Optimizer stripped from /usr/src/ultralytics/runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from /usr/src/ultralytics/runs/detect/train2/weights/best.pt, 6.2MB

Validating /usr/src/ultralytics/runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
Model summary (fused): 168 layers, 3011303 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.83it/s]


                   all          9         44      0.451      0.663      0.636      0.405
                   arc          9          2      0.464          1      0.497      0.382
                   bar          9          3          0          0     0.0603     0.0181
                button          9          3      0.409          1      0.597      0.165
          buttonmatrix          9          2      0.592          1      0.995      0.846
              calendar          9          2      0.203        0.5       0.62      0.391
              checkbox          9          5      0.666      0.399      0.746      0.354
              dropdown          9          2      0.424        0.5       0.39      0.296
                 label          9          4          0          0      0.123     0.0291
                   led          9          2      0.188        0.5      0.291      0.224
                roller          9          6      0.378      0.833      0.645       0.41
                 scal

2024-04-08 14:50:25,195 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/LVGL%20UI%20Detector/train.ebe477286dd742229e5e63b939edabca/models/best.pt


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(data=f"/workspace/project/tmp/random_widgets.yaml", epochs=100, imgsz=480, device=[0])

In [ ]:
# Perform validation using the validate function
model = YOLO('/usr/src/ultralytics/runs/detect/train7/weights/best.pt')
results = model.val()

In [ ]:
results.box.map

In [ ]:
results.box.map50

In [ ]:
results.box.map75

In [ ]:
results.box.maps

In [ ]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model='/usr/src/ultralytics/runs/detect/train7/weights/best.pt', data='/workspace/project/tmp/custom/custom.yaml', imgsz=416, half=False, device=[0])